# Promote from staging to Open Data Site

## 1 - Import python libraries

In [1]:
import copy
import getpass
import json
import os
import re
import sys
import time
import traceback
import urllib
import urllib.request as request
import urllib.request as urlopen
from datetime import datetime
import requests
from IPython.display import display
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

## 1.2 - Set up the global information and variables

In [2]:
global open_data_group
global open_data_group_prod
global online_username
global online_username_admin
global gis_online_connection

## 1.3 - Open connection to ArcGIS Online Organization

In [3]:
def connect_to_arcGIS():

    """Open connection to ArcGIS Online Organization"""
        
    online_username = input('Username: ')
    online_password = getpass.getpass('Password: ')
    online_connection = "https://www.arcgis.com"
    gis_online_connection = GIS(online_connection, 
                                online_username, 
                                online_password)
    
    return online_username, gis_online_connection

## 1.4 - IDs of staging and production ArcGIS Online Groups

In [4]:
# open_data_group_id:  
#open_data_group_stage_id = '967dbf64d680450eaf424ac4a38799ad' #Travis
open_data_group_stage_id = 'ad013d2911184063a0f0c97d252daf32'  #Luis
#open_data_group_prod_id = '15c1671f5fbc4a00b1a359d51ea6a546' # SDG Open Data
open_data_group_SYB_Topic02 = '29deadbc8f8c4ffaa4e39a1309753967' #SYB_Topic02
open_data_group_SYB_Topic03 = 'f490c0547c1441239d1e2774cff18ee1' #SYB_Topic03
open_data_group_SYB_Topic04 = 'd29d2a9106e4428d8bd3f3a0cae4e021' #SYB_Topic04
open_data_group_SYB_Topic05 = '583d7057c4ca40b28ee344dd271ffd07' #SYB_Topic05
open_data_group_SYB_Topic06 = '990a435a64044593ad5d91457af012ac' #SYB_Topic06
open_data_group_SYB_Topic07 = '6e4d253e2e944132ab72b2d8e6e0d149' #SYB_Topic07
open_data_group_SYB_Topic08 = 'd277c4d7c8a0438394bee1e98e501cb3' #SYB_Topic08
open_data_group_SYB_Topic09 = 'e682722d7cae463f84b1b50408779451' #SYB_Topic09
open_data_group_SYB_Topic11 = '4fc4d373d4574eb1bb5e06bb825dc83c' #SYB_Topic11
open_data_group_SYB_Topic13 = 'd940686a22a84aa7820c619352d3b252' #SYB_Topic13
open_data_group_SYB_Topic14 = 'f0a7f4c435bf4d54a0b1f89e268d9137' #SYB_Topic14
open_data_group_SYB_Topic15 = '692d1929a8fa4f91b16e1098afe8d451' #SYB_Topic15
open_data_group_SYB_Topic16 = '043d947fea5b467e8f6071b299394c9e' #SYB_Topic16
open_data_group_SYB_Topic19 = '247d50a1970c4310bfc8e5d37c87e789' #SYB_Topic19
open_data_group_SYB_Topic20 = '768edf7961644d29a730a47c3075c11a' #SYB_Topic20
open_data_group_SYB_Topic22 = 'a5d0dd033b3e449295276a43e77b8eaa' #SYB_Topic22
    

# 2. Main publication block

## 2.1 Connect to ArcGIS 

In [5]:
online_username, gis_online_connection = connect_to_arcGIS()
online_username_admin = 'unstats_admin'

open_data_group = gis_online_connection.groups.get(open_data_group_stage_id)
#open_data_group_prod = gis_online_connection.groups.get(open_data_group_prod_id)

open_data_group_SYB_Topic02 = gis_online_connection.groups.get(open_data_group_SYB_Topic02)
open_data_group_SYB_Topic03 = gis_online_connection.groups.get(open_data_group_SYB_Topic03)
open_data_group_SYB_Topic04 = gis_online_connection.groups.get(open_data_group_SYB_Topic04)
open_data_group_SYB_Topic05 = gis_online_connection.groups.get(open_data_group_SYB_Topic05)
open_data_group_SYB_Topic06 = gis_online_connection.groups.get(open_data_group_SYB_Topic06)
open_data_group_SYB_Topic07 = gis_online_connection.groups.get(open_data_group_SYB_Topic07)
open_data_group_SYB_Topic08 = gis_online_connection.groups.get(open_data_group_SYB_Topic08)
open_data_group_SYB_Topic09 = gis_online_connection.groups.get(open_data_group_SYB_Topic09)
open_data_group_SYB_Topic11 = gis_online_connection.groups.get(open_data_group_SYB_Topic11)
open_data_group_SYB_Topic13 = gis_online_connection.groups.get(open_data_group_SYB_Topic13)
open_data_group_SYB_Topic14 = gis_online_connection.groups.get(open_data_group_SYB_Topic14)
open_data_group_SYB_Topic15 = gis_online_connection.groups.get(open_data_group_SYB_Topic15)
open_data_group_SYB_Topic16 = gis_online_connection.groups.get(open_data_group_SYB_Topic16)
open_data_group_SYB_Topic19 = gis_online_connection.groups.get(open_data_group_SYB_Topic19)
open_data_group_SYB_Topic20 = gis_online_connection.groups.get(open_data_group_SYB_Topic20)
open_data_group_SYB_Topic22 = gis_online_connection.groups.get(open_data_group_SYB_Topic22)


Username:  gonzalezmorales_undesa
Password:  ··········


In [6]:
print(online_username_admin)
print(online_username)

unstats_admin
gonzalezmorales_undesa


In [7]:
def promote_sdg(topicId, 
                tableId=None, 
                seriesCode=None,
                property_update_only=False):
    try:
        ## Production Site Changes
        #  Search all the Items in Production Open Data Group
        user = gis_online_connection.users.get(online_username)
        admin_user = gis_online_connection.users.get(online_username_admin)
        if admin_user is None:
            return

        user_items = admin_user.items(folder='Open Data - SYB_Topic'+topicId.zfill(2), max_items=800)
        for item in user_items:
            #  Move these items into Archive folder under the Admin User
            print('Moving ' + item.title + ' to archive folder')
            item.move(folder="Historic Data SYB", owner=online_username_admin)

            #  Unshare the Items from Open Data Group (Production)
            display('unsharing item ' + item.title + " from the open data group") 
            item.unshare(open_data_group_prod["id"])
            if topicId == '2':
                item.unshare(open_data_group_SYB_Topic02["id"])
            if topicId == '3':
                item.unshare(open_data_group_SYB_Topic03["id"])
            if topicId == '4':
                item.unshare(open_data_group_SYB_Topic04["id"])
            if topicId == '5':
                item.unshare(open_data_group_SYB_Topic05["id"])
            if topicId == '6':
                item.unshare(open_data_group_SYB_Topic06["id"])
            if topicId == '7':
                item.unshare(open_data_group_SYB_Topic07["id"])
            if topicId == '8':
                item.unshare(open_data_group_SYB_Topic08["id"])
            if topicId == '9':
                item.unshare(open_data_group_SYB_Topic09["id"])
            if topicId == '11':
                item.unshare(open_data_group_SYB_Topic11["id"])
            if topicId == '13':
                item.unshare(open_data_group_SYB_Topic13["id"])
            if topicId == '14':
                item.unshare(open_data_group_SYB_Topic14["id"])
            if topicId == '15':
                item.unshare(open_data_group_SYB_Topic15["id"])
            if topicId == '16':
                item.unshare(open_data_group_SYB_Topic16["id"])
            if topicId == '19':
                item.unshare(open_data_group_SYB_Topic19["id"])
            if topicId == '20':
                item.unshare(open_data_group_SYB_Topic20["id"])
            if topicId == '22':
                item.unshare(open_data_group_SYB_Topic22["id"])

            #  Update Tags (Remove Current add Historic)
            item_properties = {}
            item_properties["tags"] = item.tags
            if 'Current' in item_properties["tags"]:
                item_properties["tags"] = item_properties["tags"].remove('Current');

            item_properties["tags"].append('Historic');
            item.update(item_properties=item_properties)

            # Mark this item as depracated
            set_content_status(update_item=item,authoratative=False)

        ##   Staging Site Changes
        #  Get all the Items in the Open Data Folder
        user_items = user.items(folder='Open Data - SYB_Topic' + topicId.zfill(2), max_items=800)
        for item in user_items:
            #Move all the CSV Files to the Open Data Folder of the Admin User
            # This will also move the Feature Service Layer!!!!
            if item.type == 'CSV':
                # Assign Item to the Admin User
                display('reassigning item ' + item.title + ' from ' + online_username + ' to ' + online_username_admin)
                item.reassign_to(online_username_admin, 'Open Data - SYB_Topic' + topicId.zfill(2) )
                
        # Update the Items in the Open Data Folder of the Admin User
        user_items = admin_user.items(folder='Open Data - SYB_Topic' + topicId.zfill(2), max_items=800)
        for item in user_items:
            # Update Sharing to Public, Share with Open Data Group     
            if item.type != 'CSV':
                display('updating sharing for item ' + item.title)
                if topicId == '2':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic02["id"], allow_members_to_edit=False)
                if topicId == '3':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic03["id"], allow_members_to_edit=False)
                if topicId == '4':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic04["id"], allow_members_to_edit=False)
                if topicId == '5':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic05["id"], allow_members_to_edit=False)
                if topicId == '6':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic06["id"], allow_members_to_edit=False)
                if topicId == '7':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic07["id"], allow_members_to_edit=False)
                if topicId == '8':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic08["id"], allow_members_to_edit=False)
                if topicId == '9':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic09["id"], allow_members_to_edit=False)
                if topicId == '11':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic11["id"], allow_members_to_edit=False)
                if topicId == '13':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic13["id"], allow_members_to_edit=False)
                if topicId == '14':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic14["id"], allow_members_to_edit=False)
                if topicId == '15':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic15["id"], allow_members_to_edit=False)
                if topicId == '16':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic16["id"], allow_members_to_edit=False)
                if topicId == '19':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic19["id"], allow_members_to_edit=False)
                if topicId == '20':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic20["id"], allow_members_to_edit=False)
                if topicId == '22':
                    item.share(everyone=True, org=True, groups=open_data_group_SYB_Topic22["id"], allow_members_to_edit=False)
                
                # Disable Editing on the Feature Service
                display('disable editing for ' + item.title)
                item_flc = FeatureLayerCollection.fromitem(item)
                update_dict2 = {"capabilities": "Query, Extract"}
                item_flc.manager.update_definition(update_dict2)

            #  Unshare from Staging Group
            display('unsharing item ' + item.title + " from the staging group") 
            item.unshare(open_data_group["id"])

            display('enabling delete protection for: ' + item.title)
            item.protect(enable=True)

            # Tag as Current
            display('updating item properties for ' + item.title)
            item_properties = dict()
            item_properties["tags"] = item.tags.append('Current')
            item.update(item_properties=item_properties)

            # Mark this item as authoratative
            display('marking item ' + item.title + " as authortative")
            set_content_status(update_item=item,authoratative=True)

    except:
        traceback.print_exc()
    return


In [8]:
def set_content_status(update_item, authoratative=True):
    sharing_url = gis_online_connection._url + "/sharing/rest/content/items/" + update_item.id + "/setContentStatus"
    sharing_params = {'f': 'json', 'token': gis_online_connection._con.token,
                        'status': 'org_authoritative' if authoratative else 'deprecated',
                        'clearEmptyFields': 'false'}
    r = requests.post(sharing_url, data=sharing_params)
    sharing_json_data = json.loads(r.content.decode("UTF-8"))

In [13]:
promote_sdg(topicId='22')

'reassigning item SYB116_2018 from gonzalezmorales_undesa to unstats_admin'

'reassigning item SYB117_2018 from gonzalezmorales_undesa to unstats_admin'

'reassigning item SYB121_2018 from gonzalezmorales_undesa to unstats_admin'

'reassigning item SYB122_2018 from gonzalezmorales_undesa to unstats_admin'

'unsharing item SYB116_2018 from the staging group'

...

'disable editing for SYB indicator: Health personnel per 1000 population'

'unsharing item SYB indicator: Health personnel per 1000 population from the staging group'

'enabling delete protection for: SYB indicator: Health personnel per 1000 population'

'updating item properties for SYB indicator: Health personnel per 1000 population'

'marking item SYB indicator: Health personnel per 1000 population as authortative'